# Look into SWAT test files

In [1]:
import numpy as np

In [2]:
from glob import glob
import re

In [3]:
from protozfits import File

In [4]:
files = glob("triggers/2025/06/0*/*")
files.sort()
len(files),files[:10],files[-10:]

(528,
 ['triggers/2025/06/01/SUB000_SWAT000_20250602T081311_SBID0000000000000000000_OBSID0000000000000000000_SUBARRAY_CHUNK000.fits.fz',
  'triggers/2025/06/01/SUB002_SWAT001_20250602T082642_SBID0000000000000000001_OBSID0000000000000000011_SUBARRAY_CHUNK000.fits.fz',
  'triggers/2025/06/01/SUB002_SWAT001_20250602T082642_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK000.fits.fz',
  'triggers/2025/06/01/SUB002_SWAT001_20250602T082858_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK001.fits.fz',
  'triggers/2025/06/01/SUB002_SWAT001_20250602T083103_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK002.fits.fz',
  'triggers/2025/06/01/SUB002_SWAT001_20250602T083301_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK003.fits.fz',
  'triggers/2025/06/01/SUB002_SWAT001_20250602T083436_SBID0000000000000000001_OBSID0000000000000000012_SUBARRAY_CHUNK000.fits.fz',
  'triggers/2025/06/01/SUB002_SWAT001_20250602T083436_SBID0000000000000000001_OBSID00000000000000

## Find the ObsIDs

In [5]:
re.findall(".+OBSID([0-9]+).+",files[0])

pattern = re.compile(r".+OBSID([0-9]+).+")

obsids = [int(pattern.findall(file)[0]) for file in files]
obsids = sorted(list(set(obsids)))
obsids

[0,
 11,
 12,
 21,
 31,
 41,
 42,
 51,
 61,
 71,
 81,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 101,
 1001,
 1002,
 1003,
 1004,
 1005,
 1101,
 1201,
 1301]

## Pick an ObsID

In [6]:
def get_files_for_obsid(obsid):
    #for file in files:
    #    print(file+"\n" if (re.findall(f".+OBSID(?:0+){obsid}_.+",file) != []) else "",end="")

    obsid_files = [file for file in files if (re.findall(f".+OBSID(?:0+){obsid}_.+",file) != []) ]
    #obsid_files
    
    ## Separate out the `SUBARRAY` files
    
    obsid_subarray = [file for file in obsid_files if "SUBARRAY" in file]
    print(f"{len(obsid_subarray):5d} subarray files")
    
    obsid_tel = sorted(list(set(obsid_files)-set(obsid_subarray)))
    print(f"{len(obsid_tel):} telescope files")

    return obsid_subarray,obsid_tel

## Look in the files

In [7]:
obsid = 1301
obsid = 1201
obsid = 1005
obsid = 97
#obsid = 81

In [8]:
obsid_subarray,obsid_tel = get_files_for_obsid(obsid)

    1 subarray files
1 telescope files


In [9]:
f_tel = File(obsid_tel[0])
f_sub = File(obsid_subarray[0])

This column (9) is not in the available message description. You are probably using an outdated version of the zfits reader...


In [10]:
len(f_sub.SubarrayEvents),len(f_tel.Triggers)

(5645993, 22663291)

In [11]:
[(evt.event_id,evt.event_time_s,evt.event_time_qns//4,
  np.array((evt.tel_ids,evt.trigger_ids)).T) for evt in list(f_sub.SubarrayEvents[5931:5951])]

[(5931,
  1748942773,
  497667135,
  array([[        20, 2649089308],
         [        21, 2651272669],
         [        24, 2652546690],
         [        22, 2651636861]], dtype=uint64)),
 (5932,
  1748942773,
  497699612,
  array([[        25, 2584377872],
         [        23, 2651601544]], dtype=uint64)),
 (5933,
  1748942773,
  497711752,
  array([[        26, 2649665078],
         [        27, 2648516614],
         [        21, 2651272670]], dtype=uint64)),
 (5934,
  1748942773,
  497796641,
  array([[        26, 2649665079],
         [        21, 2651272671]], dtype=uint64)),
 (5935,
  1748942773,
  497892786,
  array([[        26, 2649665080],
         [        27, 2648516615]], dtype=uint64)),
 (5936,
  1748942773,
  497904643,
  array([[        26, 2649665081],
         [        27, 2648516616]], dtype=uint64)),
 (5937,
  1748942773,
  497956600,
  array([[        21, 2651272672],
         [        26, 2649665082],
         [        24, 2652546691],
         [        27, 2

In [12]:
[(20000+i,tel.tel_id,tel.trigger_id,tel.trigger_time_s,tel.trigger_time_qns//4) for i,tel in enumerate(list(f_tel.Triggers[20000:20100]))]

[(20000, 23, 2651601543, 1748942773, 497597204),
 (20001, 20, 2649089308, 1748942773, 497667135),
 (20002, 21, 2651272669, 1748942773, 497667135),
 (20003, 24, 2652546690, 1748942773, 497667136),
 (20004, 22, 2651636861, 1748942773, 497667138),
 (20005, 25, 2584377872, 1748942773, 497699612),
 (20006, 23, 2651601544, 1748942773, 497699613),
 (20007, 26, 2649665078, 1748942773, 497711752),
 (20008, 27, 2648516614, 1748942773, 497711753),
 (20009, 21, 2651272670, 1748942773, 497711759),
 (20010, 26, 2649665079, 1748942773, 497796641),
 (20011, 21, 2651272671, 1748942773, 497796644),
 (20012, 26, 2649665080, 1748942773, 497892786),
 (20013, 27, 2648516615, 1748942773, 497892786),
 (20014, 26, 2649665081, 1748942773, 497904643),
 (20015, 27, 2648516616, 1748942773, 497904644),
 (20016, 21, 2651272672, 1748942773, 497956600),
 (20017, 26, 2649665082, 1748942773, 497956600),
 (20018, 24, 2652546691, 1748942773, 497956601),
 (20019, 27, 2648516617, 1748942773, 497956603),
 (20020, 22, 2651636

In [13]:
f_sub.SubarrayEvents[1],list(f_tel.Triggers[0:1010])

(EventWrapper(
     event_id=1
     trigger_type=1
     sb_id=9
     obs_id=97
     event_time_s=1748942772
     event_time_qns=3401292648
     trigger_ids=array([2651270104, 2652544012], dtype=uint64)
     tel_ids=array([21, 24], dtype=uint16)),
 [TriggerWrapper(
      tel_id=21
      trigger_id=2651270103
      trigger_type=3
      trigger_time_s=1748942772
      trigger_time_qns=3400362116
      readout_requested=False
      data_available=True
      hardware_stereo_trigger_mask=230),
  TriggerWrapper(
      tel_id=24
      trigger_id=2652544011
      trigger_type=3
      trigger_time_s=1748942772
      trigger_time_qns=3400362116
      readout_requested=False
      data_available=True
      hardware_stereo_trigger_mask=165),
  TriggerWrapper(
      tel_id=23
      trigger_id=2651598615
      trigger_type=3
      trigger_time_s=1748942772
      trigger_time_qns=3400543568
      readout_requested=False
      data_available=True
      hardware_stereo_trigger_mask=236),
  TriggerWrappe

In [14]:
# Looking at outputs, SWAT takes some time to connect to all...
f_tel = File(obsid_tel[0])
f_sub = File(obsid_subarray[0])
prev_sub_tel_ids = []
prev_s,prev_qns = 0,0
for sub in f_sub.SubarrayEvents[:10000]:
    if (len(sub.tel_ids) != len(prev_sub_tel_ids) or
        set(sub.tel_ids) != set(prev_sub_tel_ids)):
        print(sub.event_id,"Subarray tel_ids",sub.event_time_s,sub.event_time_qns,"time",sub.tel_ids)
        differ = set(sub.tel_ids.astype(int))-set(prev_sub_tel_ids)
        difftimes = (sub.event_time_s-prev_s)*1_000_000_000 + (sub.event_time_qns-prev_qns)//4
        print("  event difference", np.array(list(differ)),"times",difftimes)
        prev_sub_tel_ids = sub.tel_ids.astype(int)
    difftimes = (sub.event_time_s-prev_s)*1_000_000_000 + (sub.event_time_qns-prev_qns)//4
    print("  time difference",difftimes)
    prev_s,prev_qns = sub.event_time_s,sub.event_time_qns


This column (9) is not in the available message description. You are probably using an outdated version of the zfits reader...
0 Subarray tel_ids 1748942772 3400362116 time [21 24]
  event difference [24 21] times 1748942772850090529
  time difference 1748942772850090529
  time difference 232633
2 Subarray tel_ids 1748942772 3401489876 time [26 27]
  event difference [26 27] times 49307
  time difference 49307
3 Subarray tel_ids 1748942772 3402288368 time [21 26]
  event difference [21] times 199623
  time difference 199623
4 Subarray tel_ids 1748942772 3402643864 time [25 27]
  event difference [25 27] times 88874
  time difference 88874
5 Subarray tel_ids 1748942772 3402654936 time [26 27 21]
  event difference [26 21] times 2768
  time difference 2768
6 Subarray tel_ids 1748942772 3403615368 time [26 27]
  event difference [] times 240108
  time difference 240108
7 Subarray tel_ids 1748942772 3403631296 time [20 21]
  event difference [20 21] times 3982
  time difference 3982
8 Suba

In [15]:
from sys import maxsize
maxsize/1e9/(24*60*60)/365.25
maxsize

9223372036854775807

## Loop through the files and produce outputs

In [16]:
ord_a = ord("a")

In [17]:
obsids

[0,
 11,
 12,
 21,
 31,
 41,
 42,
 51,
 61,
 71,
 81,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 101,
 1001,
 1002,
 1003,
 1004,
 1005,
 1101,
 1201,
 1301]

In [18]:
# Order
# Channels vs key
#   0,  1,  2,  3,  4,  5,  6,  7
#  27, 25, 23, 22, 20, 21, 26, 24

key_to_order = { 27:0, 25:1, 23:2, 22:3, 20:4, 21:5, 26:6, 24:7 }


# MP revised version with test remotely, 20250701 ... not correct for files taken!!!
# Maybe the IPs are assigned in random order after boot ?
#key_to_order = { 22:0, 27:1, 21:2, 24:3, 20:4, 25:5, 26:6, 23:7 }


In [43]:
# 51 is all a's
# 61,71,81 is all a's, but high rate and few?

import sys
if  not hasattr(sys, 'ps1'):
    print(sys.argv)
    # assume last argument is obsid
    obsid = sys.argv[-1]
else:
    obsid = 51 # 1301 # 97 # 92 # 1003 # 1004 # 1005

In [ ]:
import gzip
file_times = gzip.open(f"{obsid}_times.txt.gz", "wt")
file_strings = gzip.open(f"{obsid}_strings.txt.gz", "wt")
calib = True

obsid_subarray,obsid_tel = get_files_for_obsid(obsid)

f_tel = File(ft:=obsid_tel.pop(0))
f_sub = File(fs:=obsid_subarray.pop(0))
print(ft,fs)

# In the SubarrayEvents, there is the trigger_ids, tel_ids.
#  Run through those for an event, then go through the tels, to get the times.

i = 0
sub_gen = f_sub.SubarrayEvents
tel_gen = f_tel.Triggers

# Start with first telescope record
tel = next(tel_gen)
t0 = tel.trigger_time_s-1_000_000_000 # Make it a second earlier for margin

tel_files = {}
t_prevs = {}
e_prevs = {}

n_calibs = 0
calibs = {}

#for sub in f_sub.SubarrayEvents:
while True:
    try:
        sub = next(sub_gen)
    except StopIteration:
        f_sub.close()
        if len(obsid_subarray) == 0:
            break # No more subarray files
        next_subarray = obsid_subarray.pop(0)
        print("Nextsubarray file:",next_subarray)
        f_sub = File(next_subarray)
        sub_gen = f_sub.SubarrayEvents
        sub = next(sub_gen)

        # DEBUG
        print(sub)
        

    
    #print(sub)
    sub_tel_ids=sub.tel_ids
    sub_trigger_ids = sub.trigger_ids
    #print("Subarray tel_ids",sub_tel_ids)
    #print("Subarray trigger_ids",sub_trigger_ids)
    sub_times={}

    tels = np.stack((sub_tel_ids,sub_trigger_ids)).T.tolist()

    #print("tels",tels)
    
    while len(tels):
        tel_id = tel.tel_id
        tel_idx = [tel.tel_id,tel.trigger_id]

    
        #print("***",tel_idx)
    
        if tel_idx in tels:
            #print("found tel_idx",tel_idx)
            
            # Open file if not already open
            if tel_id not in tel_files:
                tel_files[tel_id] = gzip.open(f"{obsid}_tel_{tel_id}.intervals.gz","wt")
                t_prevs[tel_id] = 0
                e_prevs[tel_id] = 0

            t_now_ns = tel.trigger_time_s*1_000_000_000 + tel.trigger_time_qns//4
            e_now = tel.trigger_id
            busy_status = not tel.data_available

            delta_ns = t_now_ns - t_prevs[tel_id]
            delta_evt = e_now - e_prevs[tel_id]

            tel_files[tel_id].write(f"{delta_ns:10} {delta_evt:10} {"1" if busy_status else "0"} {e_now:20} {t_now_ns:20}\n")

            t_prevs[tel_id] = t_now_ns
            e_prevs[tel_id] = e_now
            
            tels.remove(tel_idx)
            sub_times[tel.tel_id] = (tel.trigger_time_s-t0)*4_000_000_000+tel.trigger_time_qns

            continue

        # Could have a tel which has an event number higher than the subarray
        # => need to skip to the next subarray???
        # But without moving forward in the telescope file
        if tel.tel_id in sub.tel_ids:
            if tel.trigger_id > sub_trigger_ids[np.argwhere(sub_tel_ids==tel.tel_id)[0,0]]:
                #print(20*"v"+" TRIGGER AFTER SUB-ARRAY ??? "+20*"v")
                #file_strings.write(20*"v"+" TRIGGER AFTER SUB-ARRAY ??? "+20*"v"+"\n")
                continue
        else:
            #print(20*">"+" NON-COINCIDENT EVENT ??? "+20*">")
            #file_strings.write(20*">"+" NON-COINCIDENT EVENT ??? "+20*">"+"\n")
            # Are these at the same time as a missing packet?
            pass

        #print(">>>",tel_idx,tel.trigger_time_s,tel.trigger_time_qns)

        try:
            tel = next(tel_gen)
        except StopIteration:
            f_tel.close()
            if len(obsid_tel) == 0:
                break # No more telescope files
            next_tel = obsid_tel.pop(0)
            print("Next telescope file:",next_tel)
            f_tel = File(next_tel)
            tel_gen = f_tel.Triggers
            tel = next(tel_gen)

    sub_times = dict(sorted(sub_times.items()))
    min_times = min(sub_times.values())
    for key in sub_times.keys():
        sub_times[key] -= min_times
        sub_times[key] //= 4
    #print(sub_times)
    
    '''
    if max(sub_times.values())>9:
        print("... FAR DISTANT DELTA_T ...")
        print(sub_times)
    '''

    trig_str = "........"
    lts = list(trig_str)
    for key in sub_times.keys():
        lts[key_to_order[key]] = f"{sub_times[key]}"[-1]    
    trig_str = "".join(lts)

    #print(trig_str)
    file_times.write(trig_str+"\n")

    
    #chr(ord_a + np.array(list(sub_times.values()))//4)
    # +np.round((t4th_ns-min_time)/cnt_per_ns/clk_period).astype('int')
    trig_str = "........"
    lts = list(trig_str)
    for key in sub_times.keys():
        t_step = 2.5
        # 2.5ns spacing between deltaTs of TATS?
        lts[key_to_order[key]] = chr(ord_a + np.round(sub_times[key]/t_step).astype('int'))  
    trig_str = "".join(lts)
    
    #print(trig_str)
    file_strings.write(trig_str+"\n")

    # Try doing calibration w.r.t. 0th channel
    zeroth_chan = list(sub_times.keys())[0]
    if calib:
        if len(sub_times)==8:
            for key in sub_times.keys():
                calibs[key] = calibs.get(key, 0) + sub_times[key]-sub_times[zeroth_chan]
            n_calibs += 1
    
    if i<10000:
        print(trig_str)
    elif i%100_000==0:
        print(trig_str,i)
    elif i>1000_000_000:
        break
    #print(80*"-")
    i += 1

    1 subarray files
2 telescope files
triggers/2025/06/02/SUB002_SWAT001_20250602T162020_SBID0000000000000000005_OBSID0000000000000000051_TEL_CHUNK000.fits.fz triggers/2025/06/02/SUB002_SWAT001_20250602T162020_SBID0000000000000000005_OBSID0000000000000000051_SUBARRAY_CHUNK000.fits.fz
This column (9) is not in the available message description. You are probably using an outdated version of the zfits reader...
a.baaaai
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
baaaaaaa
b.aaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
baaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
a.aaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
abaaaaaa
aaaaaaaa
abaaaaaa
abaaaaaa
abaaaaaa
abaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
abaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
babaaaaa
babaaaaa
aaaaaaaa
aaaaaaaa
aa

In [ ]:
if calib:
    cal = {k:v/n_calibs for k,v in calibs.items()}
    print(n_calibs,cal)

In [ ]:
import pickle

In [ ]:
with open(f"{obsid}_calibs.pickle", 'wb') as handle:
    pickle.dump(cal, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
cal

In [44]:
tel

TriggerWrapper(
    tel_id=21
    trigger_id=82712290
    trigger_type=3
    trigger_time_s=1748881432
    trigger_time_qns=3036449432
    readout_requested=False
    data_available=True
    hardware_stereo_trigger_mask=133)

In [47]:
sys.getsizeof(tel)

104

In [48]:
356/24

14.833333333333334